<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Benjamin Feder, Ekaterina Levitskaya, Nathan Caplan.

**_Disclosure Review Examples & Exercises_**

This notebook provides you with information on how to prepare research output for disclosure control. It outlines how to prepare different kind of outputs before submitting an export request and gives you an overview of the information needed for disclosure review. _Please read through the entire notebook because it will separately discuss all outputs that will be flagged in the disclosure review process._

In [ ]:
# data manipulation
import pandas as pd
import numpy as np
pd.options.display.float_format = "{:.2f}".format

# database connection
from sqlalchemy import create_engine

# statistics package for calculating survey weights
from statsmodels.stats.weightstats import DescrStatsW

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Database Connection
host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = create_engine(connection_string)

# General Remarks on Disclosure Review

## Files you can export
In general, you can export any kind of file format. However, most researchers typically export tables, graphs, regression outputs and aggregated data. Thus, we ask you to export one of these types, which implies that every result you would like to export needs to be saved in either .csv, .txt or graph format.

## Jupyter notebooks are only exported to retrieve code
Unfortunately, you can't export results in a Jupyter notebook. Doing disclosure reviews on output in Jupyter notebooks is too burdensome for us. Jupyter notebooks will only be exported when the output is deleted for the purpose of exporting code. **This does not mean that you won't need your Jupyter notebooks during the export process.** 

## Documentation of code is important
During the export process, we ask you to provide the code for every output you would like to export. It is important for the ADRF staff to have the code to better understand what you exactly did. Understanding how research results are created is important in understanding your research output. Thus, it is important to document every step of your analysis in your Jupyter notebook. 

## General rules to keep in mind
A more detailed description of the rules for exporting results can be found on the class website. This is just a quick overview. You should go to the class website and read the entire guidelines (link below) before preparing your files for export. 
- The disclosure review is based on the underlying observations of your study. **Every statistic you want to export should be based on at least 10 data points at an individual level and at least 3 individual data points at an institution level**. See the example below. You must show the disclosure review team that every statistic you wish to export is based on those numbers by providing counts in your input file. 
- Document your code so the reviewer can follow your data work. Assessing re-identification risks highly depends on the context. Therefore, it is important that you provide context info with your analysis for the reviewer. When making a comments in the code, make sure not to use any individual statistic (e.g. the median is ...).
- Save the requested output with the corresponding code in your input and output folder. Make sure the code is executable. The code should exactly produce the output you requested.
- If you are exporting powerpoint slides that show project results, you have to provide the code which produces the output in the slide.
- Please export results only when they are final and you need them for your presentation or final project report.

## To-Do:
- Login to **Gitlab**.
- Read through the **documentation** link: adrf.readthedocs.io/en/latest/export_of_results/guidelines.html#documentation

# Disclosure Review Walkthrough

We will use the class data to construct our statistics and prepare them in a way so we can submit the output for the disclosure review.

**Our question of interest**: _What is the distribution of female earnings across the 2015 cohort?_

If we want to show a summary statistics on female earnings, we cannot use the outputs from `.describe()` function, as some of those statistics are represented by individual points (such as minimum, maximum, any percentiles, and median). 

Instead, we need to create _fuzzy percentiles_. For example, in order to find a fuzzy 25th percentile, we will average the 20th and 30th percentiles.

### Fuzzy percentiles

Let's walk through the code to create the fuzzy percentiles. We will use the `.quantile()` function to find the true values for some percentiles.

Let's start by getting our data.

> Note: whenever you are pulling statistics that you are interested in, remember to pull individual and institution counts (in order to be able to check whether they pass the disclosure threshold).

We will get all the relevant variables in order to find a distribution of earnings among females in the 2015 cohort:
- earnings (`salary`)
- survey weight (`wtsurvy`) - remember that data in the SDR needs to be weighted, as it is a subsample of a population
- individual ID (`refid`)
- institution ID (`sdrincd`)
- year of graduation (`sdr`) - where year is 2015
- gender (`gender`) - where gender is female

In [ ]:
# Get the relevant variables from the SDR data: 
# - earnings (salary),
# - survey weight (wtsurvy),
# - individual ID (refid),
# - institution ID (sdrincd),
# - where year of graduation (sdryr) is 2015, and
# - where gender (gender) is female.

query = '''
SELECT salary, wtsurvy, refid, sdrincd
FROM ncses_2019.nsf_sdr_2017
WHERE sdryr = '2015' 
AND gender = 'F'
'''

female_earnings = pd.read_sql(query,conn)

In [ ]:
# Show the head of a table

female_earnings.head()

Let's consult our data dictionary for the values that we can expect in the `salary` variable. 

The Survey of Doctorate Recipients (SDR) data dictionary states that `9999998` is a value reserved for a 'Logical Skip'. Therefore, we can remove those values. 

In [ ]:
# Remove the rows with the logical skip value

female_earnings = female_earnings[female_earnings['salary'] != 9999998.00]

Now we will apply the `DescrStatsW` function to calculate the weighted earnings distribution.

In [ ]:
# Apply DescrStatsW function to find the weighted salary

weighted_female_earnings = DescrStatsW(female_earnings.salary, weights=female_earnings.wtsurvy)

Let's say that we want to export the 25th, 50th, and 75th percentiles - we will need to make them fuzzy. We will do that by averaging the nearest percentiles. For example, in order to find a fuzzy 25th percentile, we will average the 20th and 30th percentiles.

Let's start by finding the following true percentiles on our weighted data:
- 20th and 30th (to create a fuzzy 25th percentile),
- 45th and 55th (to create a fuzzy 50th percentile),
- 70th and 80th percentile (to create a fuzzy 75th percentile). 

In [ ]:
# Find 20, 30, 45, 55, 70, 80 percentiles on the weighted data

true = weighted_female_earnings.quantile([.20, .30, .45, .55, .70, .80])

In [ ]:
true

Now let's average the percentiles to create fuzzy 25th, 50th, and 75th percentiles.

In [ ]:
# Find values for the fuzzy quantiles by averaging the percentiles 
# (e.g. to find 25th, average 20th and 30th, etc.)

fp_25 = str((true[.20] + true[.30])/2)
fp_50 = str((true[.45] + true[.55])/2)
fp_75 = str((true[.70] + true[.80])/2)

Let's save these fuzzy percentiles to a table.

In [ ]:
# Save in pandas dataframe

fuzzy = pd.DataFrame()
fuzzy['percentile'] = ['fuzzy_25', 'fuzzy_50', 'fuzzy_75']
fuzzy['value'] = [fp_25, fp_50, fp_75]

fuzzy

To export these fuzzy percentiles as a csv, you can use the `to_csv` function and designate the file path and the name. Here, we will call the file `fuzzy_female_earnings.csv` (the more descriptive the name of the file, the easier it is to review).

> In the file path, include `YOUR_USERNAME` to save the CSV in your home folder.

In [ ]:
fuzzy.to_csv('/nfshome/YOUR_USERNAME/fuzzy_female_earnings.csv')

In order to export these fuzzy percentiles, we need proof that the underlying counts contain at least 10 data points at the individual level and at least 3 data points at the institution level. 

For example, if you are interested in a distribution of earnings of female PhD graduates of only 1 university or if you are comparing earning distributions between 2 universities, you cannot export that statistic. Your sample should include at least 3 universities, for example, by comparing a group of at least 3 research universities versus a group of at least 3 teaching universities.

We can save the following table as `counts_for_female_earnings.csv` to designate that these counts correspond to our `fuzzy_female_earnings.csv` file.

We already pulled the data on individuals (`refid`) and institutions (`sdrincd`) from our SQL query above, so we can use it to do the counts of individuals and institutions.

In [ ]:
# Create a new dataframe for counts
counts_for_female_earnings = pd.DataFrame()

# Save the individual counts
counts_for_female_earnings['indiv_count'] = [female_earnings['refid'].nunique()]

# Save the institution counts
counts_for_female_earnings['inst_count'] = [female_earnings['sdrincd'].nunique()]

In [ ]:
counts_for_female_earnings

Save this to your home folder as a proof that the fuzzy percentiles are based on the count that passed the disclosure threshold.

In [ ]:
counts_for_female_earnings.to_csv('/nfshome/YOUR_USERNAME/counts_for_female_earnings.csv')

### Visualizations

Now let's plot a first, exploratory, histogram of the salary distribution. 

The plot call will have three outputs, which will we assign accordingly: the counts of the bin sizes, the edges of the bins, and the actual graphical image.

We will start with the **unweighted** histogram, as for disclosure review, we need to make sure that the **unweighted counts** pass the disclosure threshold (not the weighted counts).

In [ ]:
counts, edges, graph = plt.hist(female_earnings['salary'], bins = 150)
plt.xticks(rotation='vertical')

Let's take a closer look at the data, and group it into salary ranges.

To do that, let's explore the lowest and highest numbers in the salary, and let's bin the data based on that.

In [ ]:
# Sort values by lowest first

female_earnings.sort_values('salary').head()

In [ ]:
# Sort values by highest first

female_earnings.sort_values('salary', ascending=False).head()

Let's now bin our data in 5 salary ranges.

In [ ]:
# Create bin ranges
bins = [-np.inf,50000,100000,500000,1000000,5000000] 

# Create bin labels
labels = ['0 - 50K','50K - 100K','100K - 500K','500K - 1M', '1M - 5M']  

# Create a new column, `binned`, with values categorized into 5 bins defined above
female_earnings['salary_range'] = pd.cut(female_earnings['salary'], bins=bins, labels=labels)   

In [ ]:
# Show the head of the table with the new column

female_earnings.head()

Now we need to check the unweighted counts - at the individual level:

In [ ]:
individual_counts = female_earnings.groupby('salary_range')['refid'].nunique().reset_index()

In [ ]:
individual_counts

And on the institution level:

In [ ]:
institution_counts = female_earnings.groupby('salary_range')['sdrincd'].nunique().reset_index()

In [ ]:
institution_counts

XXX

XXX, it will also not pass the disclosure review as the count per this category is under 10 and for the institutions count it still under 3.

In this case, we might want to remove the outlier and also the category with 0 values. 

> Note: in other cases, when the count per bin is under 10 for individuals and under 3 for institutions, you can consider changing the sizes of the bins (moving the edges of the bins) in order to create bins which will contain more than 10 data points on the individual level and more than 3 data points on the institution level.

Let's drop `500K - 1M` and `1M - 5M` categories.

In [ ]:
female_earnings = female_earnings[(female_earnings['salary_range'] != '500K - 1M') & (female_earnings['salary_range'] != '1M - 5M')]

In [ ]:
# Remove unused categories
female_earnings['salary_range'] = female_earnings.salary_range.cat.remove_unused_categories()

Now let's check the unweighted counts at the individual level again:

In [ ]:
individual_counts = female_earnings.groupby('salary_range')['refid'].nunique().reset_index()

In [ ]:
# Rename the individual ID variable (`refid`) to `individ_counts` for better readability

individual_counts = individual_counts.rename(columns={'refid':'individ_counts'})
individual_counts

And on the institution level:

In [ ]:
institution_counts = female_earnings.groupby('salary_range')['sdrincd'].nunique().reset_index()

In [ ]:
# Rename the institution ID variable (`sdrincd`) to `inst_counts` for better readability
institution_counts = institution_counts.rename(columns={'sdrincd':'inst_counts'})
institution_counts

Let's save the unweighted counts to the "Input" folder as a disclosure review proof for our graph.

In [ ]:
all_counts = individual_counts.merge(institution_counts,on='salary_range')
all_counts

In [ ]:
all_counts.to_csv('/nfshome/YOUR_USERNAME/indiv_inst_counts_for_earnings_graph.csv')

We now have more than 10 data points on the individual level and more than 3 data points on the institution level.

> If there are less data points than needed, there are different ways to adjust the bins. You can play around with the number of bins, drop outliers, or manipulate bins so that each bin contains at least 10 entries at the individual level and at least 3 institutions by moving the `edges` of the bins.

Let's plot the unweighted histogram for the salary ranges that we are interested in.

In [ ]:
counts, edges, graph = plt.hist(female_earnings['salary'])

In [ ]:
# Check the counts per bin
counts

The last 3 bins have less than 10 data points at the individual level. We can combine the last 4 bins together.

In [ ]:
# Let's take a look at the edges of the bins
edges 

In [ ]:
# Now let's adjust the bins based on the edges above (combine the last 4 bins)

counts, edges, graph = plt.hist(female_earnings['salary'], bins = [0, 29808, 59616, 89424,
                                                                  119232, 149040,178848,
                                                                  298080])

In [ ]:
# Now let's check the counts of individuals again
counts

The counts of individuals are now fine. Now we need to check the counts of institutions.

In [ ]:
# Use the bin ranges defined above 
bins = [-np.inf,29808, 59616, 89424,119232, 149040,178848,298080]

# Create bin labels
labels = ['0 - 30K','30K - 60K','60K - 89K','89K - 120K','120K - 149K','149K - 179K', '179K - 300K']  

# Create a new column, `binned`, with values categorized into 5 bins defined above
female_earnings['new_bins'] = pd.cut(female_earnings['salary'], bins=bins, labels=labels)   

Now let's check the number of unique institutions for every bin.

In [ ]:
inst_count = female_earnings.groupby('new_bins')['sdrincd'].nunique().reset_index()

In [ ]:
inst_count = inst_count.rename(columns={'sdrincd':'inst_count'})
inst_count

Let's add individual counts from above to this table, and save all counts as a proof.

In [ ]:
inst_count['indiv_count'] = counts

In [ ]:
inst_count

We need to save this unweighted histogram and the unweighted counts as a disclosure review proof.

In [ ]:
# Rename inst_count to all_counts and save in the "Input" folder

all_counts.to_csv('/nfshome/YOUR_USERNAME/counts_for_unweighted_hist_female_earnings.csv')

In [ ]:
# Save the unweighted histogram with new bins
plt.hist(female_earnings['salary'], bins = [0, 29808, 59616, 89424,
                                            119232, 149040,178848,298080])
plt.savefig('/nfshome/YOUR_USERNAME/unweighted_hist_female_earnings.pdf')

For the weighted histogram, you must keep the same bins as above.

In [ ]:
counts, edges, graph = plt.hist(female_earnings['salary'], 
                                weights = female_earnings['wtsurvy'], 
                                bins = [0, 29808, 59616, 89424, 119232, 
                                        149040,178848,298080])

You can save it now in the home folder, as we already saved the unweighted histogram and unweighted counts as proof that the counts pass the disclosure threshold.

In [ ]:
plt.hist(female_earnings['salary'], 
        weights = female_earnings['wtsurvy'], 
        bins = [0, 29808, 59616, 89424, 119232, 
                149040,178848,298080])

plt.savefig('/nfshome/YOUR_USERNAME/weighted_hist_female_earnings.pdf')

Now we can also produce a bar plot - based on the weighted counts, as we are working with a subsample of data.

In [ ]:
# Sum the weights per salary range

female_earnings_grouped = female_earnings.groupby('salary_range')['wtsurvy'].sum().reset_index().set_index('salary_range')

In [ ]:
female_earnings_grouped

In [ ]:
female_earnings_grouped.plot(kind='bar', rot='horizontal', fontsize = 12)

Finally, we can save the weighted bar plot in the home folder.

In [ ]:
female_earnings_grouped.plot(kind='bar', rot='horizontal', fontsize = 12)

plt.savefig('/nfshome/YOUR_USERNAME/exports/weighted_barplot_female_earnings.pdf')

Now you have successfully saved your output!

Please keep in mind that any other statistics that you are using (e.g. for counts of subpopulations) have to comply with the disclosure threshold described above -  we need not only the **counts of individuals** (**more than 10**), but also the **counts of institutions** (**more than 3**).

### Machine Learning

Whenever you are creating your training and test datasets, after creating them, please include the counts of each variable (individual and institution counts), and please do not alter the datasets afterwards. 

Remember that if you are plotting y-scores, it is still a histogram, and each estimate represents an individual data point, therefore, it needs to comply with the disclosure threshold described above.

For the precision, you can use the outputs, however, we still need to know the value counts of the training and test datasets.

### Reminder
Every single item you wish to export, regardless of whether it is a .csv, .pdf, .png, or something else, must have corresponding proof in your input file to show that every group used to create this statistic followed our disclosure review rules.

> Note: After the end of the course, you can export the code that you have been using. In order to do that, you will need to clear the outputs of the notebooks.

<font color=red><h3> Checkpoint: Do your first export</h3></font>

You should already have some descriptives statistics or visualizations that you can export. Do your first export using the rules and code above.